In [1]:
# Import Dependencies
import requests
from config import api_key
import censusdata
import pandas as pd  
from sqlalchemy import create_engine
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
import numpy as np
from sqlalchemy import func
from sqlalchemy import distinct
import json
import psycopg2

In [2]:
cloud_engine = create_engine(f"postgresql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
cloud_conn = cloud_engine.connect()

ca_school_data = pd.read_sql('select * from ml_master_clean', cloud_conn)

ca_school_data.head()

,Zipcode,Total Students Enrolled,Median Household Income,Median Income BLACK/AA,Median Income INDIGENOUS PEOPLE,Median Income ASIAN,Median Income HAW/PAC ISLANDER,Median Income OTHER RACE,Median Income MIXED RACE,Median Income WHITE/NOT LATINO,...,Substance Abuse Count,Violent Count,Miscellaneous Count,School Name,School Rating,School Address,City,Grades,Students Per Teacher,District
0,91901,244.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,The Heights Charter,7.0,"2710 Alpine Boulevard, Suite E",Alpine,K-8,15:1,Dehesa Elementary
1,91901,333.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Boulder Oaks Elementary,6.0,2320 Tavern Road,Alpine,1-5,24:1,Alpine Union Elementary
2,91901,438.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Joan MacQueen Middle,6.0,2001 Tavern Road,Alpine,6-8,20:1,Alpine Union Elementary
3,91901,387.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Shadow Hills Elementary,6.0,8770 Harbison Canyon Road,Alpine,K-5,20:1,Alpine Union Elementary
4,91901,158.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Alpine Elementary,4.0,1850 Alpine Boulevard,Alpine,1-5,20:1,Alpine Union Elementary


In [3]:
ca_school_data.columns

Index(['Zipcode', 'Total Students Enrolled', 'Median Household Income',
       'Median Income BLACK/AA', 'Median Income INDIGENOUS PEOPLE',
       'Median Income ASIAN', 'Median Income HAW/PAC ISLANDER',
       'Median Income OTHER RACE', 'Median Income MIXED RACE',
       'Median Income WHITE/NOT LATINO', 'Median Income HISPANIC/LATINO',
       'o25_10th_grade_attained_ratio', 'o25_11th_grade_attained_ratio',
       'o25_12th_grade_started_ratio', 'o25_high_school_completed_ratio',
       'o25_ged_attained_ratio', 'o25_less_one_yr_college_ratio',
       'o25_more_one_yr_college_ratio', 'o25_assoc_degree_completed_ratio',
       'o25_bach_degree_completed_ratio', 'o25_masters_degree_completed_ratio',
       'o25_professional_degree_completed_ratio',
       'o25_doctoral_degree_completed_ratio', 'married ratio',
       'female hoh ratio', 'male hoh ratio', 'non family ratio',
       'Population Total', 'Population white ratio',
       'Population black/aa ratio', 'Population indigenous 

In [4]:
ca_school_drop = ca_school_data.drop(labels=[
       'Median Income BLACK/AA', 'Median Income INDIGENOUS PEOPLE',
       'Median Income ASIAN', 'Median Income HAW/PAC ISLANDER',
       'Median Income OTHER RACE', 'Median Income MIXED RACE',
       'Median Income WHITE/NOT LATINO', 'Median Income HISPANIC/LATINO',
       'Theft Count', 'latitude', 'longitude', 'Substance Abuse Count', 
       'Violent Count', 'Miscellaneous Count', 'School Name', 
       'School Address', 'City', 'Grades', 'District'], axis=1)
ca_school_drop.head(75)

,Zipcode,Total Students Enrolled,Median Household Income,o25_10th_grade_attained_ratio,o25_11th_grade_attained_ratio,o25_12th_grade_started_ratio,o25_high_school_completed_ratio,o25_ged_attained_ratio,o25_less_one_yr_college_ratio,o25_more_one_yr_college_ratio,...,Population white ratio,Population black/aa ratio,Population indigenous ratio,Population asian ratio,Population haw/pac islander ratio,Population other ratio,Population mixed ratio,Population hispanic/latino ratio,School Rating,Students Per Teacher
0,91901,244.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,7.0,15:1
1,91901,333.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,24:1
2,91901,438.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,20:1
3,91901,387.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,20:1
4,91901,158.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,4.0,20:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,91915,332.0,101760.0,0.008781,0.002117,0.023789,0.131959,0.006165,0.052248,0.176174,...,0.215569,0.058173,0.000674,0.208302,0.000000,0.010777,0.033961,0.472544,4.0,14:1
71,91916,211.0,69821.0,0.006108,0.020942,0.006981,0.166667,0.015707,0.099476,0.263525,...,0.838610,0.006828,0.000000,0.014277,0.004345,0.000000,0.015518,0.120422,6.0,21:1
72,91932,896.0,46659.0,0.022927,0.017494,0.036062,0.187116,0.032360,0.087408,0.208430,...,0.334589,0.030465,0.002975,0.082282,0.008887,0.000226,0.039578,0.500998,6.0,20:1
73,91932,1585.0,46659.0,0.022927,0.017494,0.036062,0.187116,0.032360,0.087408,0.208430,...,0.334589,0.030465,0.002975,0.082282,0.008887,0.000226,0.039578,0.500998,4.0,25:1


In [5]:
# new data frame with split value columns
new = ca_school_drop["Students Per Teacher"].str.split(":", n = 1, expand = True)
  
# making separate first name column from new data frame
ca_school_drop["Students Per Teacher Count"]= new[0]
#ca_school_drop

ca_school_updated = ca_school_drop.drop(labels=["Students Per Teacher"], axis=1)
ca_school_updated

,Zipcode,Total Students Enrolled,Median Household Income,o25_10th_grade_attained_ratio,o25_11th_grade_attained_ratio,o25_12th_grade_started_ratio,o25_high_school_completed_ratio,o25_ged_attained_ratio,o25_less_one_yr_college_ratio,o25_more_one_yr_college_ratio,...,Population white ratio,Population black/aa ratio,Population indigenous ratio,Population asian ratio,Population haw/pac islander ratio,Population other ratio,Population mixed ratio,Population hispanic/latino ratio,School Rating,Students Per Teacher Count
0,91901,244.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,7.0,15
1,91901,333.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,24
2,91901,438.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,20
3,91901,387.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,20
4,91901,158.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,4.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,92672,422.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,9.0,18
692,92672,575.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,7.0,25
693,92672,929.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,7.0,26
694,92672,889.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,5.0,27


In [6]:
ca_school_updated.to_csv('sd_schools_tableau.csv',index=False)

In [7]:
# Using OneHotEncoder to create the features
# Generate our categorical variable lists
ca_school_cat = list(ca_school_updated.dtypes[ca_school_updated.dtypes == "object"].index)
ca_school_cat

['Students Per Teacher Count']

In [8]:
ca_school_updated["Students Per Teacher Count"] = ca_school_updated["Students Per Teacher Count"].astype(str).astype(int)
ca_school_updated

,Zipcode,Total Students Enrolled,Median Household Income,o25_10th_grade_attained_ratio,o25_11th_grade_attained_ratio,o25_12th_grade_started_ratio,o25_high_school_completed_ratio,o25_ged_attained_ratio,o25_less_one_yr_college_ratio,o25_more_one_yr_college_ratio,...,Population white ratio,Population black/aa ratio,Population indigenous ratio,Population asian ratio,Population haw/pac islander ratio,Population other ratio,Population mixed ratio,Population hispanic/latino ratio,School Rating,Students Per Teacher Count
0,91901,244.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,7.0,15
1,91901,333.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,24
2,91901,438.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,20
3,91901,387.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0,20
4,91901,158.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,4.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,92672,422.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,9.0,18
692,92672,575.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,7.0,25
693,92672,929.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,7.0,26
694,92672,889.0,77829.0,0.007934,0.004768,0.010939,0.122135,0.009978,0.062831,0.190335,...,0.710389,0.003197,0.001528,0.030981,0.000594,0.000764,0.025181,0.227365,5.0,27


In [9]:
ca_school_cat = list(ca_school_updated.dtypes[ca_school_updated.dtypes == "object"].index)
ca_school_cat

[]

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split our preprocessed data into our features and target arrays
X = ca_school_updated.drop("School Rating", axis=1).values
y = ca_school_updated["School Rating"].values.reshape(-1, 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

model = linear_model.LinearRegression()

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

# The coefficients
print("Coefficients: \n", model.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))


Coefficients: 
 [[ 0.01526408  0.13877642  0.31095271 -0.01111653 -0.11273739 -0.00643411
   0.3532923   0.08751751  0.04292489  0.05947374  0.02302843  0.44267972
   1.17582516  0.38075128 -0.49420418 -0.03453944  0.14400541 -0.24217062
   0.02612478  0.16944535 -0.16531016 -0.03491382 -0.12753463  0.08938923
  -0.04909811 -0.03873834  0.05724885  0.16085205  0.25075365]]
Mean squared error: 2.45
Coefficient of determination: 0.47


In [13]:
model_2 = linear_model.LinearRegression()

model_2.fit(X_train_scaled, y_train)

y_pred_2 = model_2.predict(X_train_scaled)

# The coefficients
print("Coefficients: \n", model_2.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_pred_2))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_train, y_pred_2))

Coefficients: 
 [[ 0.01526408  0.13877642  0.31095271 -0.01111653 -0.11273739 -0.00643411
   0.3532923   0.08751751  0.04292489  0.05947374  0.02302843  0.44267972
   1.17582516  0.38075128 -0.49420418 -0.03453944  0.14400541 -0.24217062
   0.02612478  0.16944535 -0.16531016 -0.03491382 -0.12753463  0.08938923
  -0.04909811 -0.03873834  0.05724885  0.16085205  0.25075365]]
Mean squared error: 2.66
Coefficient of determination: 0.43


In [14]:
y_true = y_test
df = pd.DataFrame({'true':y_true.flatten(),'pred':y_pred.flatten()})
df['MSE'] = (df['pred']-df['true'])**2
df['SD'] = np.sqrt((df['MSE']))
print('Model: Linear Regression')
print('R-squared: %.3f' % model.score(X_test_scaled, y_test))
df.describe()

Model: Linear Regression
R-squared: 0.468


,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.764368,5.848120,2.445768,1.244031
std,2.149525,1.424955,3.370928,0.950445
min,2.000000,3.409437,0.000018,0.004194
25%,4.000000,4.824787,0.249913,0.499912
50%,6.000000,5.473637,1.132959,1.064372
75%,8.000000,6.965024,2.984143,1.727467
max,10.000000,9.754828,18.915802,4.349230
